# Model ResNet

https://www.pluralsight.com/guides/introduction-to-resnet

#### TODOS
1. DONE Debugging, does output make sense?
    1. Resize images
    2. preprocessing fixes
    5. replace scaling by proper function
2. try on leaderboard
3. Include Odometry and fuse into heads
    - Speed
    - Location
4. navigation
5. controller
6. Evaluation on Test set, Modularization



## Dependencies

In [1]:
# MODEL STUFF
import torch
import torch.nn as nn
import torch.optim as optim
#import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import *
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms

# GENERAL STUFF
import time
import copy
import os
import sys
sys.path.insert(1, 'C:\\Users\\morit\\OneDrive\\UNI\\Master\\WS22\\APP-RAS\\Programming\\data_pipeline') # TODO

# DATA ENGINEERING
from data_sampler import WeightedSampler
from dataset import CARLADataset#, CARLADatasetMultiProcessing

## Model

In [2]:
class MyResnet(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        # ResNet Architecture with pretrained weights, also bigger resnets available
        self.net = torchvision.models.resnet18(weights=True)
        num_ftrs = self.net.fc.in_features

        # Top layer of ResNet which you can modify. We choose Identity to use it as Input for all the heads
        self.net.fc = nn.Identity()
        
        # Input Layer fuer cmd, spd
        self.cmd_input = nn.Sequential(
            nn.Linear(7, 7),
            nn.Tanh() #nn.LeakyReLU() # TODO
        )
        
        self.spd_input = nn.Sequential(
            nn.Linear(1, 1),
            nn.Tanh() #nn.LeakyReLU() # TODO
        )
        
        # MLP
        self.mlp = nn.Sequential(
            nn.Linear(num_ftrs+8, num_ftrs+8),
            nn.Tanh(), #nn.LeakyReLU()
            #nn.Dropout(p=0.5, inplace=False),
            nn.Linear(num_ftrs+8, 200),
            nn.Tanh(), #nn.LeakyReLU()
            #nn.Dropout(p=0.5, inplace=False),
            nn.Linear(200, 100),
            nn.Tanh(), #nn.LeakyReLU()
            #nn.Dropout(p=0.3, inplace=False),
            nn.Linear(100, 30),
            nn.Tanh(), #nn.LeakyReLU()
            #nn.Dropout(p=0.2, inplace=False),
            nn.Linear(30, 10),
            nn.Tanh(), #nn.LeakyReLU()
            #nn.Dropout(p=0.2, inplace=False)
        )
        
        # Regression Heads for Throttle, Brake and Steering
        self.thr_head = nn.Sequential(
            nn.Linear(10, 1),
            nn.Sigmoid() # [0,1] Range Output
        )
        
        self.brk_head = nn.Sequential(
            nn.Linear(10, 1),
            nn.Sigmoid() # [0,1] Range Output
        )
        
        self.str_head = nn.Sequential(
            nn.Linear(10, 1),
            nn.Tanh() # [-1,1] Range Output
        )

    # Forward Pass of the Model
    def forward(self, rgb, cmd, spd):
        rgb = self.net(rgb) # BRG
        cmd = self.cmd_input(cmd)
        spd = self.spd_input(spd)
        
        x = torch.cat((rgb, cmd, spd),1)
        x = self.mlp(x)
        
        #x = self.net.fc(x)
        return self.thr_head(x), self.str_head(x), self.brk_head(x) # 3 Outputs since we have 3 Heads


## Data Loaders, Data Sets

In [3]:
#path_ege_data = os.path.join("..", "..", "data", "Dataset Ege")
train_path = "D:\\data\\Train"
test_path = "D:\\data\\Test"

config = {"used_inputs": ["rgb","measurements"], 
        "used_measurements": ["speed", "steer", "throttle", "brake", "command"],
        "seq_len": 1
        }

train_dataset = CARLADataset(root_dir=train_path, config=config)
test_dataset = CARLADataset(root_dir=test_path, config=config)

weighted_sampler = WeightedSampler(dataset=train_dataset)

batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [4]:
len(train_dataloader)

372

In [5]:
len(test_dataloader)

41

In [6]:
augumentations = torch.nn.ModuleList([
        transforms.GaussianBlur(9),
        transforms.ColorJitter(brightness=1.0, contrast=0.5, saturation=1, hue=0.1),
        transforms.RandomErasing()
    ])

In [7]:
#mean = torch.tensor([79.6657, 81.5673, 105.6161]) BGR
#std = torch.tensor([66.8309, 60.1001, 66.2220])

mean = torch.tensor([105.6161, 81.5673, 79.6657]) # RGB
std = torch.tensor([66.2220, 60.1001, 66.8309])


transform_norm = transforms.Compose([
    transforms.Normalize(mean, std),
    transforms.Resize([224,224])
])

transform_augument = transforms.RandomApply(augumentations)

## Training

In [8]:
# Initialise Model (GPU or CPU)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net = MyResnet().cuda() if device else net
net

C:\Users\morit\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MyResnet(
  (net): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runnin

In [9]:
def to_cuda_if_possible(data):
    return data.to(device) if device else data

In [10]:
def forward_pass(data, augument = True):
    # further preprocessing
    X_rgb = transform_norm(torch.squeeze(data["rgb"])).float()
    if False:#augument:
        X_rgb = transform_augument(X_rgb)
    labels = data["command"]
    labels = torch.where(labels == -1, torch.tensor(0), labels).to(torch.int64) # Replace by -1 by 0
    # Convert the labels to a one hot encoded tensor
    one_hot = torch.nn.functional.one_hot(labels, num_classes=7)
    X_cmd = torch.squeeze(one_hot).float()
    X_spd = ((data["speed"]-speed_mean)/speed_std).float()
    
    Y_throttle = data["throttle"].float()
    Y_steer = data["steer"].float()
    Y_brake = data["brake"].float()

    # move to GPU
    X_rgb = to_cuda_if_possible(X_rgb)
    X_cmd = to_cuda_if_possible(X_cmd)
    X_spd = to_cuda_if_possible(X_spd)
    
    Y_throttle = to_cuda_if_possible(Y_throttle)
    Y_steer = to_cuda_if_possible(Y_steer)
    Y_brake = to_cuda_if_possible(Y_brake)

    # compute outputs
    optimizer.zero_grad()

    Y_hat = net(X_rgb, X_cmd, X_spd)
    Y_hat_throttle = to_cuda_if_possible(Y_hat[0])
    Y_hat_steer = to_cuda_if_possible(Y_hat[1])
    Y_hat_brake = to_cuda_if_possible(Y_hat[2])

    # get labels from data
    Y_throttle = to_cuda_if_possible(data["throttle"].float())
    Y_steer = to_cuda_if_possible(data["steer"].float())
    Y_brake = to_cuda_if_possible(data["brake"].float())

    # Calculate Loss
    loss_throttle = criterion(Y_hat_throttle, Y_throttle)
    loss_steer = criterion(Y_hat_steer, Y_steer)
    loss_brake = criterion(Y_hat_brake, Y_brake)
    loss = sum([loss_throttle, loss_steer, loss_brake])/3
    return loss

In [11]:
# Loss and Optimizer
criterion = nn.L1Loss()# nn.MSELoss()##  # Easy to interpret #
optimizer = optim.Adam(net.parameters(), lr=0.0001) #,weight_decay=1e-5 #optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

## Model Trainer Prototype

In [12]:
speed_mean = 3.5797309975940315#2.382234#2.250456762830466
speed_std = 0.13709178245795187#1.724884#0.30215840254891313

In [13]:
%%time

n_epochs = 1
print_every = 50
valid_loss_min = np.Inf
val_loss = []
train_loss = []
total_step = len(train_dataloader)

nan_batches = []

run = True

for epoch in range(1, n_epochs+1):
    
    running_loss = 0.0
    print(f'Epoch {epoch}\n')
    
    # Work through batches
    for batch_idx, data in enumerate(train_dataloader): #data: (['idx', 'rgb', 'speed', 'steer', 'throttle', 'brake'])

        loss = forward_pass(data)
        
        # Backprop
        loss.backward()
        optimizer.step()
        
        loss_value = loss.item()
        
        """
        if np.isnan(loss_value):
            print("nan", batch_idx)
            nan_batches.append((batch_idx, data))
         """   
        
        running_loss += loss_value
        if (batch_idx) % print_every == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch, n_epochs, batch_idx, total_step, loss_value ))
        
    # Epoch finished, evaluate network and save if network_learned
    train_loss.append(running_loss/total_step)
    print(f'\ntrain-loss: {np.mean(train_loss):.4f},') # TODO SOLVE NAN ISSUES
    batch_loss = 0

    
    # Evaluation on Test set, skipped for now

    with torch.no_grad():
        net.eval()
        
        for batch_idx, data in enumerate(test_dataloader):
            
            loss = forward_pass(data, False)
            
            if (batch_idx) % print_every == 0:
                print ('Validation [{}/{}], Step [{}/{}], Loss: {:.4f}'
                       .format(epoch, n_epochs, batch_idx, total_step, loss_value ))
            
            batch_loss += loss.item()
        val_loss.append(batch_loss/len(test_dataloader))
        #network_learned = batch_loss < valid_loss_min
        print(f'validation loss: {np.mean(val_loss):.4f}, \n') # TODO SOLVE NAN ISSUES

        
        if False:#network_learned:
            valid_loss_min = batch_loss
            torch.save(net.state_dict(), 'resnet.pt')
            print('Improvement-Detected, save-model')

    # Back to training
    net.train()

Epoch 1

Epoch [1/1], Step [0/372], Loss: 0.1234
Epoch [1/1], Step [50/372], Loss: 0.0599
Epoch [1/1], Step [100/372], Loss: 0.0574
Epoch [1/1], Step [150/372], Loss: 0.0565
Epoch [1/1], Step [200/372], Loss: 0.0575
Epoch [1/1], Step [250/372], Loss: 0.0478
Epoch [1/1], Step [300/372], Loss: 0.0455
Epoch [1/1], Step [350/372], Loss: 0.0471

train-loss: 0.0569,
Validation [1/1], Step [0/372], Loss: 0.0439
validation loss: 0.1453, 

Wall time: 11min 10s


### Test predictions

In [29]:
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
iterator = iter(test_dataloader)
#print(next(iter(test_dataloader)).keys())

In [ ]:
data = next(iterator)
#data

X_rgb = transform_norm(torch.squeeze(data["rgb"])).float().to(device)
labels = data["command"]
labels = torch.where(labels == -1, torch.tensor(0), labels).to(torch.int64) # Replace by -1 by 0
# Convert the labels to a one hot encoded tensor
one_hot = torch.nn.functional.one_hot(labels, num_classes=7).to(device)
X_cmd = torch.squeeze(one_hot).float().to(device)
X_spd = ((data["speed"]-speed_mean)/speed_std).float().to(device)
print(np.mean(X_spd.cpu().numpy()))

target_ = (data["throttle"], data["steer"], data["brake"])
with torch.no_grad():
    net.eval()
    outputs_ = net(X_rgb, X_cmd, X_spd)
    
# Durchschnittlicher abs. fehler
for i in [0,1,2]:
    print(np.mean(abs(outputs_[i].cpu().numpy()-target_[i].cpu().numpy())))

-8.845869


In [ ]:
print()

In [ ]:
np.round(outputs_[0].cpu().numpy()-target_[0].cpu().numpy(),2)

Bias Variance

In [ ]:
# Variance 

for i in [0,1,2]:
    outputs = (outputs_[i].cpu().numpy())
    #print(outputs)
    mean_outputs = np.mean(outputs_[i].cpu().numpy())
    #print(mean_outputs)
    diff = (outputs-mean_outputs)**2
    #print(diff)
    value = np.mean(diff)
    print(value)

In [ ]:
# Bias
for i in [0,1,2]:
    targets = (target_[i].cpu().numpy())
    #print(outputs)
    mean_outputs = np.mean(outputs_[i].cpu().numpy())
    #print(mean_outputs)
    diff = outputs-mean_outputs
    #print(diff)
    value = np.mean(diff)
    print(value)

In [ ]:

for i in [0,1,2]:
    print(np.mean(abs(target_[i].cpu().numpy())))
    print(np.std(abs(target_[i].cpu().numpy())))


In [ ]:
i =0

In [ ]:
print(np.round(outputs_[i].cpu().numpy(),1))

In [ ]:
print(np.round(target_[i].cpu().numpy(),1))

### IMG Processing

BGR is now standard FOR carla agent and training

In [ ]:
import cv2
idx, batch = next(enumerate(test_dataloader))
print(batch["rgb"].shape)

In [ ]:
img = batch["rgb"][0]#.shape
img = img.numpy().astype(np.uint8).reshape(160,960,3)
print(img.shape)

#img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # NUR HIER, NICHT IN CARLA AGENT
print(img.shape)
print(type(img))
transform = transforms.Compose([transforms.ToPILImage()])

tensor = transform(img)

#print(type(tensor))

tensor.show()

#torch.tensor(tensor)

TEST Normalization

In [ ]:
tensor = transform_norm(torch.squeeze(data["rgb"],1))

In [ ]:
tensor = torch.squeeze(transform_norm(data["rgb"])).float()

In [ ]:
for i in range(64):
    print(np.mean(tensor.numpy()[i], axis = (1,2)))

In [ ]:
np.mean(tensor.numpy(), axis = (0,2,3))

### adding Navigation and speed

In [ ]:
test_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
iterator = iter(test_dataloader)



In [ ]:
data = next(iterator)
#data["speed"]
#data["command"]

Command

In [ ]:
import torch

# Assume labels is a 1D tensor with values from 0 to 6

labels = data["command"]
labels = torch.where(labels == -1, torch.tensor(0), labels) # Replace by -1 by 0
labels = labels.to(torch.int64)

# Convert the labels to a one hot encoded tensor
one_hot = torch.nn.functional.one_hot(labels, num_classes=7)
one_hot = torch.squeeze(one_hot)

print(one_hot.shape)

Speed

In [ ]:
# calc mean over trainingsset
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
print(len(train_dataloader))
iterator = iter(train_dataloader)

In [ ]:
i = 0
summe = []
for batch_idx, data in enumerate(train_dataloader):
    #print(data)
    if i % 100 == 0:
        print(i)
    summe.append(np.mean(data["speed"].numpy()))
    i += 1
    if i >= 1000:
        break


In [ ]:
#print(summe)

In [ ]:
print(np.mean(summe)) # 2.2078979146598274
print(np.std(summe)) # 0.22455625005948113
speed_mean = np.mean(summe)
speed_std = np.std(summe)

In [ ]:
batch = next(iterator)
#print(np.round(batch["speed"].numpy(),2))

In [ ]:
(batch["speed"]-speed_mean)/speed_std

In [ ]:
np.mean(((batch["speed"]-speed_mean)/speed_std).numpy())

### Vanishing/Exploding Gradients

In [ ]:
"""
for name, param in net.thr_head.named_parameters():
    if param.requires_grad:
        print(name, param.data.cpu().numpy())
"""

In [ ]:
for name, param in net.spd_input.named_parameters():
    if param.requires_grad:
        print(name, np.max(abs(param.data.cpu().numpy())))

## Saving and Loading

Not suited for leaderboard agents

In [ ]:
#torch.save(net, 'rgb_resnet.pth')

In [ ]:
#net = torch.load('rgb_resnet.pth')

suited for leaderboard agents

In [ ]:
#torch.save(net.state_dict(), "rgb_resnet_cmd_spd_bgr_fix_3E.pth")

In [ ]:
#net = MyResnet()
#net.load_state_dict(torch.load("rgb_resnet_cmd_spd_bgr_fix_3E.pth"))
#net.cuda()

## Testing Time

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
idx, X = next(enumerate(test_dataloader))
img = transform_norm(X["rgb"])
img.shape

In [ ]:
torch.squeeze(img,1).shape

In [ ]:
# ohne preprocessing ca 16-17 sekunden. Mit preprocessing ca 37 sekunden ~2gb
# 24.12: 44 batches -> preprocessing 26 sec, training & preprocessing 69 sec
#45
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
print(len(test_dataloader))
at = time.time()
for batch_idx, data in enumerate(test_dataloader):
    pass
    #print(batch_idx)
    #data_ = torch.squeeze(transform_norm(data["rgb"]),1).float()
    #print(data_.shape)
et = time.time()
print(et-at)
